# 表形式ファイルの処理

## やること

pandasを使ってRNA-seqデータの処理をします

1. サンプルごとのカウントデータを1つのテーブルにまとめる
2. データの読み込み
3. カウントデータの正規化
4. サンプル間のクラスタリング　
5. 遺伝子のアノテーション
    - transcript_idとgene_idの対応
    - gene_idに対応したdescriptionの付与
6. 発現変動遺伝子の抽出

## 使用データ

自閉症モデルマウス細胞のRNA-seqデータセット

Impaired KDM2B-mediated PRC1 recruitment to chromatin causes defective neural stem cell self-renewal and ASD/ID-like behaviors
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE190806

BioProject	PRJNA788503
- SRA	SRP350646; SRR17223720-SRR17223725
- SRR17223720-2 Neural progenitor cells(C57BL/6J由来), **wild-type**
- SRR17223723-5 Neural progenitor cells(C57BL/6J由来), **Kdm2b one allele mutant**


## ディレクトリとファイルの説明

- `3/data/kallisto/` kallistoによるサンプルごとのカウントデータが入っている
- `3/data/SRR_Acc_list.tsv` サンプルリスト
- `3/data/annotation.tsv`　遺伝子アノテーション（遺伝子名とIDの対応）
- `3/output` これから計算する結果を入れていく


【参考】
- 実験医学別冊　独習Pythonバイオ情報解析
- pandas 公式サイト　https://pandas.pydata.org
- note.nknk.me pandas関連記事まとめ　https://note.nkmk.me/python-pandas-post-summary/
- kallisto を用いた A. thaliana paired-end リードの転写産物の定量 https://bi.biopapyrus.jp/rnaseq/mapping/kallisto/kallisto-paired.html
- Quasi-Mappingによって高速にRNA seqの定量を行う Kallisto https://kazumaxneo.hatenablog.com/entry/2018/07/14/180503
- kallisto Manual http://pachterlab.github.io/kallisto/manual.html


## 1. カウントテーブルの作成

- サンプルごとのカウントデータを1つのカウントテーブルにまとめる

kallistoではサンプル1個ごとのカウントデータが得られます<br>
ほかのツールで処理するときは全部のデータをまとめたほうが扱いやすいので、kallistoのカウント結果 `abundance.tsv` をまとめてひとつのカウントテーブルを作ります<br>
具体的には、kallistoで出力した`effective_length`と`estimate_count`でTPMを算出するため、結果を全部結合して`estimate_count`を残す、という作業をします


必要なモジュールをインポートします

In [ ]:
import sys
import pandas as pd
import numpy as np

# versionの確認
print(sys.version)
print(pd.__version__)	# pandasのバージョン確認
print(np.__version__)	# numpyのバージョン確認

SRAアクセッションを1行ずつ並べた `SRR_Acc_List.txt` を読み込みます

In [ ]:
sralib=[i[:-1] for i in open('data/SRR_Acc_List.txt','r')]
sralib = tuple(sralib) # 順番を変えたくないのでtupleにする

In [ ]:
# 確認
sralib

kallistoによるカウント結果　abundance.tsvのPATHのリストを作ります<br>
`data`フォルダの下の`kallisto`フォルダに、それぞれの結果フォルダが入っています

In [ ]:
kallisto_counts=[]
for sra in sralib:
    kallisto_counts.append('data/kallisto/' + sra + '_exp_kallisto/abundance.tsv')

kallisto_counts = tuple(kallisto_counts) # 順番を変えたくないのでtupleにする

確認します

In [ ]:
kallisto_counts

`abundance.tsv`を読み込んで、<br>
`estimate_count`列と`tpm`列にSRAアクセッションを追加する処理を、<br>
`read_countdata()`という関数にしておきます

In [ ]:
def read_countdata(num):
    df = pd.read_table(kallisto_counts[num],sep='\t')
    newcol1 = 'est_counts_' + sralib[num]
    newcol2 = 'tpm_' + sralib[num]
    df.rename(columns = {'est_counts':newcol1,'tpm':newcol2}, inplace=True)

    return df


カウントデータを読み込みます

In [ ]:
df0 = read_countdata(0)
df1 = read_countdata(1)
df2 = read_countdata(2)
df3 = read_countdata(3)
df4 = read_countdata(4)
df5 = read_countdata(5)

読み込めているか確認します（いずれも同じ行数になるはず）

In [ ]:
print(len(df0),len(df1),len(df3),len(df3),len(df4),len(df5))

df0の最初の5行を表示してみます

In [ ]:
df0.head()

`target_id`列（transcript_id）と`length`列, `eff_rength`列（TPMの計算に使う）は最初のサンプルの分だけでいいので、残りの表から削除します

In [ ]:
df1_count = df1.copy().drop(columns=['length','eff_length'])
df2_count = df2.copy().drop(columns=['length','eff_length'])
df3_count = df3.copy().drop(columns=['length','eff_length'])
df4_count = df4.copy().drop(columns=['length','eff_length'])
df5_count = df5.copy().drop(columns=['length','eff_length'])

target_idをkeyとして、すべてつなげます

In [ ]:
new_df = pd.merge(df0, df1_count, on = 'target_id')
new_df = pd.merge(new_df, df2_count, on = 'target_id')
new_df = pd.merge(new_df, df3_count, on = 'target_id')
new_df = pd.merge(new_df, df4_count, on = 'target_id')
new_df = pd.merge(new_df, df5_count, on = 'target_id')

確認します

In [ ]:
new_df.head()

`est_counts`と`eff_length` だけのtable、`tpm` だけのtableを作ります<br>
列名もSRAアクセッションのみにします

In [ ]:
# est_countsとeff_length
drop_column1 = ['tpm_'+ i for i in sralib]
drop_column1.append('length')
rename_column1 = {'est_counts_'+ i:i for i in sralib}
new_df_count = new_df.copy().drop(columns = drop_column1).rename(columns=rename_column1)

確認します

In [ ]:
new_df_count.head()

In [ ]:
# tpm
drop_column2 = ['est_counts_'+ i for i in sralib]
drop_column2.append('length')
drop_column2.append('eff_length')
rename_column2 = {'tpm_'+ i:i for i in sralib}
new_df_tpm = new_df.copy().drop(columns = drop_column2).rename(columns=rename_column2)

確認します

In [ ]:
new_df_tpm.head()

タブ区切りファイルとして保存します

In [ ]:
new_df_count.to_csv('data/counts_kallisto.tsv', sep="\t",index=False)
new_df_tpm.to_csv('data/tpm_kallisto.tsv', sep="\t",index=False)

## 2.データファイルの読み込み

In [ ]:
# カウントデータをまとめたファイル ( counts_kalisto.tsv ) のパスを指定
count_file = 'data/counts_kallisto.tsv'

#### カウントデータファイルについて<br>
counts_kallisto.tsvを開くと以下のようになっています<br>
```
target_id	eff_length	SRR17223720	SRR17223721	SRR17223722	SRR17223723	SRR17223724	SRR17223725
ENSMUST00000178537.2	6.74193	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000178862.2	7.65825	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000196221.2	5.34639	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000179664.2	6.27959	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000177564.2	8.56364	0.0	0.0	0.0	0.0	0.0	0.0
... 以下省略
```
1行目は列タイトルを表すヘッダー行です<br>
2行目以降からがデータ行です。一番左の列が遺伝子idになっているのでこれをインデックスに用います

`pd.read_table()` メソッドの`index_col`オプションを指定して読み込みます<br>

- index_col...インデックスとして用いる列を数字で指定します

In [ ]:
df = pd.read_table(count_file, index_col=0)

#### データの概観
データを概観してみましょう

In [ ]:
# head()で列名とインデックスが正しく読み込まれているかを確認します
df.head()

In [ ]:
# 'eff_length'列だけ確認
df.eff_length.head()

In [ ]:
# データ件数を確認
df.shape

#### 列名を変更する

In [ ]:
# 列名を変更するための対応表
names = {'SRR17223720': 'wt_1',
         'SRR17223721': 'wt_2',
         'SRR17223722': 'wt_3',
         'SRR17223723': 'mutant_1',
         'SRR17223724': 'mutant_2',
         'SRR17223725': 'mutant_3'}

`rename()` を `axis=1` を適用して使い、列名を変更します

In [ ]:
df = df.rename(mapper=names, axis=1)

In [ ]:
# 列名が変更されたことを確認
df.head()

保存用に`output/`フォルダを用意し, estimate_countのみのデータを`count_raw.tsv`として保存します

In [ ]:
df_count = df[['wt_1','wt_2','wt_3','mutant_1','mutant_2','mutant_3']]

# estimate_countのみのデータの保存
df_count.to_csv('output/count_raw.tsv', sep = '\t')

## 3. カウントデータの正規化
### Step1. リード数で正規化 (RPM/FPM)

100万リードあたりのカウント数に揃えます<br>
RPM = reads per million, FPM = fragments per million<br>, ほぼ同じ意味で用いられているので, 本講義では「FPM」を使います<br>
<br>
カウントデータをいったん別のデータフレームとしてコピーしておきます

In [ ]:
df_tmp = df_count.copy()

In [ ]:
# リード数の合計　sum()を使って計算
sum_count = df_tmp.sum()

In [ ]:
sum_count

In [ ]:
# 100万リードあたりに揃える
df_tmp = 10**6 * df_tmp / sum_count

リード数の合計が100万に揃っていることを確認します

In [ ]:
df_tmp.sum()

`normalize_per_million_reads()` として関数化しておきます

In [ ]:
def normalize_per_million_reads(df):
    sum_count = df.sum()
    return 10**6 * df / sum_count

カウントデータに適用

In [ ]:
df_count_fpm = normalize_per_million_reads(df_count)

In [ ]:
#  確認
df_count_fpm.sum()

### Step2. 遺伝子長による正規化 (RPKM/FPKM)
上で求めたFPMをさらに遺伝子長で割ってFPKMを求めます<br>
FPKM = fragments per kilobase of exon per million reads mapped <br>
今回用いたsingle-endの場合、<br>
RPKM = reads per kilobase of exon per million reads mapped と呼ばれますが, <br>
FPKM/RPKMはほぼ同じ意味で用いられています

各遺伝子の長さを抽出しておきます<br>
kallistoでは実際の配列長（abundance.tsvの`length`列）ではなく、`eff_length`列でTPMを計算しているので、<br>
本講義でも`eff_length`列の値を用います<br>


【参考】　次世代シーケンサーデータの解析手法 第15回 RNA-seq 解析(その3)<br>
DOI: https://doi.org/10.4109/jslab.31.25


In [ ]:
gene_length = df['eff_length']

本でいくつか計算方法を紹介していますが、今日はデータフレームを転置してから計算する方法でやってみます

In [ ]:
# テスト用にFPMをコピー
df_tmp = df_count_fpm.copy()

# df_tmpを転置してFPMを遺伝子長で割り, 1000をかける
df_tmp = df_tmp.T / gene_length * 10**3

# 戻す（もう一度転置する）
df_tmp = df_tmp.T

これを `normalize_per_kilobase()` として関数化しておきます

In [ ]:
def normalize_per_kilobase(df, gene_length):
    df_tmp = df.copy()
    df_tmp = (df.T * 10**3 / gene_length).T
    return df_tmp

先ほど計算したFPMの結果に適用します

In [ ]:
df_count_fpkm = normalize_per_kilobase(df_count_fpm, gene_length)

In [ ]:
# 保存する
df_count_fpkm.to_csv('output/count_fpkm.tsv', sep='\t')

### Step3. TPM 正規化
TPM = transcripts per million (transcripts per kilobase million)<br> 
TPM の説明については以下のページが詳しいです https://bi.biopapyrus.jp/ <br> 

FPKM/RPKM のときとは逆に、長さ1,000bpあたりのリード数を求めてから、総リード数を100万に揃えます

In [ ]:
# テスト用にカウントデータをコピー
df_tmp = df_count.copy()

In [ ]:
df_tmp = normalize_per_kilobase(df_tmp, gene_length) #長さ1,000bpあたりのリード数
df_tmp = normalize_per_million_reads(df_tmp) #総リード数を100万に揃える

RPKM/FPKMと違い、合計が100万となっています

In [ ]:
df_tmp.sum()

`normalize_tpm()` として関数化しておきます

In [ ]:
def normalize_tpm(df, gene_length):
    df_tmp = df.copy()
    df_tmp = normalize_per_kilobase(df_tmp, gene_length)
    df_tmp = normalize_per_million_reads(df_tmp)
    return df_tmp

カウントデータに適用します

In [ ]:
df_count_tpm = normalize_tpm(df_count, gene_length)

In [ ]:
# 確認
df_count_tpm.sum()

In [ ]:
# 保存
df_count_tpm.to_csv('output/count_tpm.tsv', sep='\t')

### 4. TPM正規化したデータのクラスタリング
matplotlib と scipy の必要モジュールをインポートします

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
# 転置
tpm_t = df_count_tpm.T

# 確認
tpm_t.iloc[:,0:3]

In [ ]:
# クラスタリング
linkage_result = linkage(tpm_t, method='average', metric='correlation')

# 結果の可視化
plt.figure(num=None, figsize = (8,4), facecolor='w', edgecolor='k')
dendrogram(linkage_result,labels = list(tpm_t.index) )
plt.show()

PCAもやってみましょう

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(tpm_t)

# データを主成分空間に写像
pca_row = pca.transform(tpm_t)

# 寄与率を求める
pca_col = ["PC{}".format(x + 1) for x in range(len(tpm_t.index))]
df_con_ratio = pd.DataFrame([pca.explained_variance_ratio_], columns = pca_col)
print(df_con_ratio.head())

# PC1とPC2でplot
plt.figure(figsize=(4, 4))
plt.scatter(pca_row[:, 0], pca_row[:, 1], alpha=0.8, c=(1,1,1,2,2,2))
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
annotations = tpm_t.index
for i, label in enumerate(annotations):
    plt.annotate(label, (pca_row[i, 0], pca_row[i, 1]))
plt.show()

## 5. 遺伝子のアノテーション

#### Step1. アノテーションファイルを読み込む

アノテーションデータ `annotation.tsv` はGRCm39のGFFファイル( `Mus_musculus.GRCm39.108.gff3` ) から必要な情報を抽出して作成しました<br>
`annotation.tsv`を開くと以下のようになっています<br>

```
ENSMUST00000070533.5	ENSMUSG00000051951	Xkr4	X-linked Kx blood group related 4 [Source:MGI Symbol;Acc:MGI:3528744]
ENSMUST00000208660.2	ENSMUSG00000025900	Rp1	retinitis pigmentosa 1 (human) [Source:MGI Symbol;Acc:MGI:1341105]
ENSMUST00000027032.6	ENSMUSG00000025900	Rp1	retinitis pigmentosa 1 (human) [Source:MGI Symbol;Acc:MGI:1341105]
ENSMUST00000027035.10	ENSMUSG00000025902	Sox17	SRY (sex determining region Y)-box 17 [Source:MGI Symbol;Acc:MGI:107543]
ENSMUST00000195555.2	ENSMUSG00000025902	Sox17	SRY (sex determining region Y)-box 17 [Source:MGI Symbol;Acc:MGI:107543]
...
```

In [ ]:
# アノテーションデータ(transcript_id, gene_id, descriptionを紐づけたファイル ( annotation.tsv ) のパスを指定
annotation_file = 'data/annotation.tsv'

DataFrame名を `annotations` として、 <br>
ヘッダー（列名）は `names=['transcript_id', 'gene_id', 'gene_name', 'description']`として読み込みます

In [ ]:
annotations = pd.read_table(annotation_file, names=['transcript_id', 'gene_id', 'gene_name', 'description'])
print(annotations.shape)
annotations.head()

#### Step2. アノテーションデータとカウントデータを連結
2つのDataFrame `df` と `annotations` を連結します<br>
`annotation` には mRNA のデータしか含まれていないので、rRNA などのデータはこの時点で除かれます<br>
ただし、`gene_name`列の値が `mt-` で始まるミトコンドリア遺伝子が含まれています

In [ ]:
df_with_product = df.copy()
df_with_product['transcript_id'] = df_with_product.index
df_with_product.head()

pandasの`merge()`を使います

In [ ]:
df_with_product = pd.merge(df_with_product, annotations, on='transcript_id', how='inner')

In [ ]:
# transcript_id列をindexにする
df_with_product.set_index('transcript_id', inplace=True)

アノテーション付きカウントデータを`count_preprocessed.tsv`として保存します<br>

In [ ]:
# アノテーション付きデータの保存
df_with_product.to_csv('output/count_preprocessed.tsv', sep = '\t')

TPMデータにもアノテーション付与

In [ ]:
df_tpm_with_product = df_count_tpm.copy()
df_tpm_with_product['transcript_id'] = df_tpm_with_product.index
df_tpm_with_product = pd.merge(df_tpm_with_product, annotations, on='transcript_id', how='inner')
df_tpm_with_product.set_index('transcript_id', inplace=True)

df_tpm_with_product.head()

In [ ]:
# アノテーション付きデータの保存
df_tpm_with_product.to_csv('output/tpm_with_product.tsv', sep = '\t')

## 6. 発現変動遺伝子を抽出する


TPM正規化データを用います<br>
wild typeの平均を求めます

In [ ]:
df_count_tpm['wt'] = (df_count_tpm['wt_1'] + df_count_tpm['wt_2'] + df_count_tpm['wt_3']) / 3

mutantの平均を求めます

In [ ]:
df_count_tpm['mutant'] = (df_count_tpm['mutant_1'] + df_count_tpm['mutant_2'] + df_count_tpm['mutant_3']) / 3

発現変動をlog2 fold として求めます<br>
0 での除算を防ぐため、分母に微小な値を加えています<br>

In [ ]:
df_count_tpm['log2fold'] = df_count_tpm['mutant'] / (df_count_tpm['wt'] + 10**-6)
df_count_tpm['log2fold'] = df_count_tpm['log2fold'].apply(np.log2)

必要部分のみ抜き出し、遺伝子アノテーション（タンパク質名）と結合します<br>
`df_count_tpm` から'wt', 'mutant', 'log2fold'の列を抜き出し`diff_ex`とします

In [ ]:
diff_ex = df_count_tpm.copy()
diff_ex = diff_ex[['wt', 'mutant', 'log2fold']]
diff_ex ['transcript_id']= diff_ex.index

`annotations`（遺伝子アノテーションのテーブル）と結合します

In [ ]:
diff_ex = pd.merge(diff_ex, annotations, on = 'transcript_id', how = 'right')
diff_ex.set_index('transcript_id', inplace = True)

In [ ]:
diff_ex.head()

カウント数が0であるデータを除きます

In [ ]:
diff_ex = diff_ex[diff_ex['wt'] > 0]
diff_ex = diff_ex[diff_ex['mutant'] > 0]

In [ ]:
# データ数の確認
diff_ex.shape

In [ ]:
# 概観
diff_ex.head()

`diff_ex.sort_values()`を使い`log2fold`の降順に並びかえます

In [ ]:
diff_ex = diff_ex.sort_values(by = 'log2fold',ascending = False)

発現変動遺伝子の上位を表示してみましょう

In [ ]:
# mutant > wt の上位5番目まで表示
diff_ex.head()

In [ ]:
# wt > mutant の上位5番目まで表示
diff_ex.tail()

おつかれさまでした！